Refresher
--------------------

We hope that you are somewhere safe. 

To get you back into Computational Journalism, this notebook reminds you about Pandas and plotly and all the fun we were having before the world fell apart. Along the way, you'll see some things you know well (or knew well, and will know well again) and some new tricks. 

**A quick look at mapping**

As Mike noted on Slack, the New York Times made their [county-level COVID-19 data available to the public here.](https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv) These are the data that are underneath their [graphic tracking the virus.](https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html) Let's see if we can recreate their map!

The data are released as a CSV (what does CSV stand for?). Preview it on GitHub using the link above. You'll see the columns include a date field, county and state names and then the number of COVID19 cases and the number of people who have died from the virus as of the specified date. So each row refers to the statistics from a particular county on a given day.

The data also include a column for the county's "FIPS code." FIPS stands for Federal Information Processing Standards and [the FIPS code uniquely identified places in the US, like states and counties and cities.](https://en.wikipedia.org/wiki/FIPS_county_code) Think of it as a unique identifier -- while we have over 30 states with a Washington county in the US, instead of having to keep track of the state as well as the county, we can uniquely identify specific counties using the single FIPS code. Handy.

So, as you remember, step one is simply to read the CSV file (what does CSV stand for?) into Python as a Pandas Data Frame. Below we import the `read_csv()` function and load up the `us-counties.csv` file from the Times' GitHub using its URL. Again, each row of the table records the number of COVID19 cases and deaths in a county as of a particular day.

Note that in the code below we use an argument `dtype` for `read_csv()` to specify the data type for specific columns. The column name and desired data type is specified as a dictionary. 

In this case, we want to make sure that the column `fips` be read as a string. This is because, for exmample, LA County's FIPS code is 06037. If we read it as a number, it might become 6,037, losing the leading 0. That might be a problem when we compare FIPS codes from different data sets. So to be careful, we want to keep the leading 0 and read the FIPS code as a character string "06037". 

In [ ]:
from pandas import read_csv

covid_map = read_csv("https://github.com/nytimes/covid-19-data/raw/master/us-counties.csv",dtype={"fips":"str"})
covid_map.head()

Remember that the method `sort_values()` reorders the rows according to the values of a variable or variables in the Data Frame. Here we order the table so that those counties with the highest counts appear at the top (again, counties might appear multiple times because each row in the table contains data for a county on a given day). 

In [ ]:
covid_map.sort_values("cases",ascending=False)

Notice that the top counties included "New York City". This is a little sloppy, but  basically, the Times collapsed the five counties that make up NYC and referred to them as "New York City". For the purpose of making a bubble plot like the Times did, we will assign the New York City measurements a FIPS code of 36061 -- that of Manhattan. So the bubble will be draw in the middle of Manhattan. From the scale of a country-wide map, it should be fine and (while sloppy from a data perspective) certainly better than drawing five large circles on top of each other in essentially the same place.

So to make this change, the code below uses the `loc[ ]` function to select rows and columns for "New York City" and assign them the FIPS code of 36061. Remember `loc[ ]`? It specifies rows and columns at one time, separating our choices with a comma. So below we take the rows where the county is "New York City" and the column "fips". 

In [ ]:
covid_map.loc[covid_map["county"]=="New York City","fips"]

So as we saw above, the FIPS codes are all missing. Here we replace them with Manhattan's code. 

In [ ]:
covid_map.loc[covid_map["county"]=="New York City","fips"] = "36061"

And have a look...

In [ ]:
covid_map.loc[covid_map["county"]=="New York City","fips"]

To make the bubble map that the Times created, we need to associate the county-level case counts with a location, the center of the county. The bubble is located over the center of the county, sized according to the number of cases. To do this, we will `merge()` the `covid_map` Data Frame with another data set where each row is a county and the columns include FIPS code and the latitude and longitude of the center of the county. 

We found this other data set on the web and tossed it up on our course GitHub site. It's a CSV -- let's read it in! [Preview it here](https://github.com/computationaljournalism/columbia2020/raw/master/data/county_FIPS.csv). 

We read it in as we did the Times' data, also being careful with the FIPS column.

In [ ]:
counties = read_csv("https://github.com/computationaljournalism/columbia2020/raw/master/data/county_FIPS.csv",dtype={"fips":"str"})
counties.head()

The `.merge()` method of a Data Frame allows us to add information from one data set, expanding another. Here we will add data to `covid_map`. In the code below, for each row of `covid_map` we will take its `fips` code and look it up in the Data Frame `counties`. When it finds a match, it adds the county name (just for a visual check) and the latitude and longitude of the center of the county to the row of `covid_map` DataFrame. Do this for every row of `covid_map` and you end up with 3 new columns. `merge()` is very useful.

The `on` argument in the call to `merge()` below, says that `fips` will be the "key" used to match a county in `covid_map` with the data in `counties` -- both Data Frames have a column caled `fips`. If the names of the columns with the FIPS codes were different in each Data Frame we could use the arguments `left_on` and `right_on` to specify the column names for `covid_map` and `counties`, respectively.  

The function `merge()` is quite powerful and allows different techniques for joining two data sets. We'll go over more of the possibilities later. 

OK for `covid_map` and `counties` we do the following...

In [ ]:
covid_map = covid_map.merge(counties,on="fips")
covid_map.head()

Finally, we can make the bubble plot! In `plotly.express` there is a command called `scatter_geo`. It takes a Data Frame, looks for the latitude and longitude of each point, associates a column in the data set with the size of the bubbles, and even provides for simple animation -- in this case, we can animate day-by-day.

In [ ]:
from plotly.express import scatter_geo

fig = scatter_geo(covid_map,
                      lat = "lat",            # the latitude of the center of a county
                      lon = "lon",            # the longitude of the center of a county
                      hover_name="county",    # the name of a county shows up on "hover"
                      size="cases",           # size the bubble by the number of cases in a county
                      size_max=50,            # I eyeballed how big the biggest bubble should be
                      animation_frame="date", # let the map animate by day
                      scope="usa")            # make a map of the us, not the world or europe
fig

The map will be empy because it starts showing data in January and there was not much to see. Hit the play button or just move the dot on the slider to see how things change in time.

**Country curves**

Next, we are going to work with Johns Hopkins COVID data. They are updating a data set every day. You can [read about their collection efforts here](https://github.com/CSSEGISandData/COVID-19). You can [preview the data set here.](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv)

Let's load it in.

In [ ]:
from pandas import read_csv

covid = read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
covid.head()

There is one big difference between this data set and the one the Times released? What do you notice about the structure Johns Hopkins chose? 

For the Times, each row represents cases in a county on a given day. For Johns Hopkins, each row is a place and date is encoded in columns. Each new day adds another column (scroll to the right above to see this).

The form the Times used is a little simpler to work with. We can easily create a time series for a given place by subsetting rows (asking for all the rows with FIPS code "36061" to get the cases over time from New York City. But with Johns Hopkins' data, the time series (curve) for cases by day is represeted as a row in the table. None of our code is really built to work with that -- usually graphs give columns that are x and y. 

So we can reformat this data using a DataFrame method called `melt()`. It will take the Data Frame from its current "wide" form to a "long" form. Let's do it and then talk about it...

In [ ]:
new_covid=covid.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Cases')
new_covid.head()

What we've done is to use the Province/State, Country/Region, Lat and Long as "ID variables". We then repeat a given place as many times as we have dates in the data set. 

So, for example, in the old Data Frame `covid` from Johns Hopkins, the US had one row. All of the case counts for each day were in different columns. The count on 1/22/20 is in its own column, as is the count on 3/27/20.

In [ ]:
covid[covid["Country/Region"]=="US"]

Now, after `melt()`ing, we have the US data in separate rows, one row per date. So the count on 1/22/20 is in its own row and the count on 3/27/20 is in its own row.

In [ ]:
new_covid[new_covid["Country/Region"]=="US"]

The command `melt()` wants you to assign `id_vars` which in this case specifies the place the data come from (its name and lat/long) and then the `value_name` you'd like to stretch out from columns into rows (here, we chose the name `Cases` because that's what the data in each row of `covid` represents -- the data that aren't referring to the place itself) and finally the `var_name` which indicates the data represented by the different columns (here we made up the name "Date", as the columns referred to dates -- again the data not associated with the place itself like its name and lat/long).

Now that the data is in a form we can work with, we need to do one more thing. We would like to have cases per Country per day. Right now, some places like the US are just a single row, whereas China is broken up in to provinces. So what we will do is `groupby()` the data into groups made up of Country/Region and Date pairs. Then, for each group we will sum the number of cases and produce a total daily count per Country/Region. We've seen `groupby()` before, but this is a good refresher.

In [ ]:
new_covid=new_covid.groupby(['Country/Region','Date'])['Cases'].sum().reset_index()
new_covid

Now, the date fields are strings. They are also the bad kind of string. "3/2/20" comes before "3/10/20" for example. So we won't be able to just sort the data by the Date field to order things chronologically. Instead we use `to_datetime` that can guess the date format and create a date "object" that will sort corerctly. 

In the code below, we import the function from `pandas` and then create a new column in `new_covid` called "new_date" to hold the datetime objects rather than the janky strings. We also `sort_values()` to sort the data into increasing order by "new_date".  

In [ ]:
from pandas import to_datetime

new_covid["new_date"] = to_datetime(new_covid["Date"])
new_covid = new_covid.sort_values("new_date")

new_covid.head()

See the extra column?

Finally, a plot! We've seen `line()` from `plotly.express` before. Or maybe just the scatterplot. In any event, we can now trace the epidemic in, say, the US. Here is the US data...

In [ ]:
new_covid[new_covid["Country/Region"]=="US"]

And here we use that same data set in `line()`. We just have to specify the x- and y-axes.

In [ ]:
from plotly.express import line
fig = line(new_covid[new_covid["Country/Region"]=="US"],x="new_date",y="Cases")
fig

Now we will make a plot for all the places. We will use one line per Country/Region. In `line()` we assign the "color" argument the name of the column "Country/Region" to be used to color separate lines, one for each place. 

In [ ]:
from plotly.express import line
fig = line(new_covid,x="new_date",y="Cases",color="Country/Region")
fig

**API review and open-ended work**

Now, some open-ended work. Open this URL: https://covidtracking.com/api/us/daily.csv. It represents a CSV. In this case each row is a day and then for each day, among other things, we record the number of people who tested positive and negative for COVID19 in the US up to that day. These data are part of the [COVID Tracking Project](https://covidtracking.com/) maintained by Alexis Madrigal.

When we downloaded `daily.csv` we were, in fact, making an API call. That same API could return for us just the data from New York state, say --https://covidtracking.com/api/states/daily.csv?state=NY. Or all the data from California from last Sunday -- https://covidtracking.com/api/states/daily.csv?state=NY&date=20200322.

These data are unique because they are tracking the number of tests being given and their results. Given the warmup above and some possible review you might need from old notebooks, work with this data a little. What stories might be here to pursue? At very least, stretch your programming legs and read in a Data Frame or two and tell me what you see.

In [ ]:
# your code here
